<a href="https://colab.research.google.com/github/Snezaa/Snezaa.github.io/blob/main/%D0%92%D0%BD%D0%B5%D1%88%D0%BD%D0%B8%D0%B5_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5_%D0%BB%D0%BE%D0%BA%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%84%D0%B0%D0%B9%D0%BB%D1%8B%2C_%D0%94%D0%B8%D1%81%D0%BA%2C_%D0%A2%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%8B_%D0%93%D0%B5%D1%82%D0%BC%D0%B0%D0%BD%D0%BE%D0%B2%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом блокноте представлены способы загрузки и сохранения данных из внешних источников.

# Локальная файловая система

## Загрузка файлов из локальной файловой системы

Метод <code>files.upload</code> возвращает словарь загруженных файлов.
Для словаря используется доступ по ключу в виде названия файла, а в качестве значений выступают загруженные данные.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Гетманова.xlsx to Гетманова.xlsx
User uploaded file "Гетманова.xlsx" with length 12163 bytes


## Скачивание файлов в локальную файловую систему

Метод <code>files.download</code> активирует скачивание файла из браузера на локальный компьютер.


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Google Диск

Доступ к файлам на Диске можно получить несколькими способами, например:
- Подключение Google Диска к виртуальной машине в среде выполнения
- C помощью оболочки для API, такой как <a href="https://docs.iterative.ai/PyDrive2/">PyDrive2</a>.
- Использование <a href="https://developers.google.com/drive/v3/web/about-sdk">нативного REST API</a>



Примеры использования каждого метода приведены ниже.

## Локальное подключение Google Диска

В примере ниже показано, как подключить Google Диск к среде выполнения с помощью кода авторизации, а также как записывать и читать файлы. После выполнения кода новый файл &#40;<code>foo.txt</code>&#41; будет доступен на странице <a href="https://drive.google.com/">https://drive.google.com/</a>.

Этот параметр можно использовать только для чтения, записи и перемещения файлов. Чтобы изменять настройки доступа на программном уровне, используйте один из других вариантов ниже.

<strong>Примечание.</strong> При использовании кнопки "Подключить Диск" в диспетчере файлов для блокнотов, которые редактировал только текущий пользователь, не нужны коды аутентификации.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Привет, Google Диск!')
!cat /content/drive/My\ Drive/foo.txt

Привет, Google Диск!

In [ ]:
drive.flush_and_unmount()
print('Все изменения, внесенные в этом сеансе colab, теперь должны быть видны на диске.')

Drive not mounted, so nothing to flush and unmount.
Все изменения, внесенные в этом сеансе colab, теперь должны быть видны на диске.


## PyDrive2

Ниже приведены примеры использования PyDrive2 для аутентификации и загрузки/скачивания файлов.

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Выполните аутентификацию и создайте клиент PyDrive2.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Создайте и загрузите текстовый файл.


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1nG3_1aPsBqzbyfsO7_Tj-6Qi18baxpfX


Загрузите файл по идентификатору и распечатайте его содержимое.


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## Drive REST API

Чтобы использовать Drive API, сначала нужно выполнить аутентификацию и создать клиент API.


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

С этим клиентом мы можем использовать любые функции из <a href="https://developers.google.com/drive/v3/reference/">документации по Google Drive API</a>. Примеры представлены ниже.


### Создание файла на Диске на основе данных из Python

Прежде всего, создайте локальный файл для загрузки.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Загрузите его, используя метод <a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a>.

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1qbgxPhW0uadCXK8v5PMDA_n19dK0kAyh


После выполнения кода в ячейке выше на странице <a href="https://drive.google.com/">https://drive.google.com/</a> появится новый файл под названием Sample file.

### Скачивание данных из файла на Диске в Python

Скачайте файл, который мы загрузили выше.

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


Чтобы скачать другой файл, приведите переменную <code>file&#95;id</code> в соответствие с его идентификатором, таким как: 1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz.

# Google Таблицы

В примерах ниже для взаимодействия с Google Таблицами используется библиотека <a href="https://github.com/burnash/gspread"><code>gspread</code></a> с открытым исходным кодом.

Импортируйте библиотеку, выполните аутентификацию и установите связь с Таблицами.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Ниже приведены примеры операций с библиотекой <code>gspread</code>.

## Создание новой таблицы с данными из Python

In [ ]:
sh = gc.create('My cool spreadsheet')

После выполнения кода в ячейке выше на странице <a href="https://sheets.google.com/">https://sheets.google.com/</a> появится новая таблица под названием My cool spreadsheet.

Откройте новую таблицу и добавьте в нее случайные данные.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1glYxJlr0XgtTiia3A2sgqvf3ranOZ9AP3cJWUCAMN7g',
 'updatedRange': "'Лист1'!A1:C2",
 'updatedRows': 2,
 'updatedColumns': 3,
 'updatedCells': 6}